1.定义工具


In [1]:
import os
from langchain_community.tools.tavily_search import TavilySearchResults
# 定义 AVILY_KEY 密钥
os.environ["TAVILY_API_KEY"] = "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"
# 查询 Tavily 搜索 API
search = TavilySearchResults(max_results=1)
# 执行查询
res = search.invoke("今天上海天气怎么样")
print(res)

[{'url': 'https://www.weather.com.cn/weather/101020100.shtml', 'content': '首页\n预报\n预警\n雷达\n云图\n天气地图\n专业产品\n资讯\n视频\n节气\n我的天空\n\n全国>\n上海\n>\n城区\n\n18:00更新 | 数据来源 中央气象台\n\n 今天\n 7天\n 8-15天\n 40天\n 雷达图\n\n # 16日（今天）\n # 17日（明天）\n\n  阴转多云\n\n  15℃/7℃\n\n  <3级\n # 18日（后天）\n\n  晴\n\n  16℃/7℃\n\n  <3级\n # 19日（周一）\n\n  多云转小雨\n\n  13℃/2℃\n\n  <3级\n # 20日（周二）\n\n  阴\n\n  3℃/0℃\n\n  <3级\n # 21日（周三）\n\n  多云\n\n  2℃/0℃\n\n  <3级\n # 22日（周四）\n\n  多云转晴\n\n  4℃/0℃\n\n  <3级\n\n分时段预报\n生活指数\n\n蓝天预报\n\n### 蓝天预报综合天气现象、能见度、空气质量等因子，预测未来一周的天空状况。\n\n 天空蔚蓝\n  可见透彻蓝天，或有蓝天白云美景。\n 天空淡蓝\n  天空不够清澈，以浅蓝色为主。\n 天空阴沉\n  阴天或有雨雪，天空灰暗。\n 天空灰霾\n  出现霾或沙尘，天空灰蒙浑浊。\n\n23时02时05时08时\n\n10℃10℃8℃9℃\n\n东风东北风东北风东北风\n\n<3级<3级<3级<3级\n\n 易发\n  感冒指数\n\n  大幅度降温，湿度较大，注意防护。\n 较适宜\n  运动指数\n\n  天气凉，在户外运动请注意增减衣物。\n 极不易发\n  过敏指数\n\n  无需担心过敏，可放心外出，享受生活。\n 较冷\n  穿衣指数\n\n  建议着厚外套加毛衣等服装。)\n 适宜\n  洗车指数\n\n  天气较好，适合擦洗汽车。\n 强\n  紫外线指数\n\n  涂擦SPF大于15、PA+防晒护肤品。\n\n 少发\n  感冒指数\n\n  无明显降温，感冒机率较低。\n 较适宜\n  运动指数\n\n  气温较低，在户外运动请注意增减衣物。\n 极不易发\n  过敏指数 [...] 昼夜温

2.定义检索器

In [3]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os
import dotenv
dotenv.load_dotenv()
# 1. 提供一个大模型
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
os.environ['OPENAI_BASE_URL'] = os.getenv("OPENAI_BASE_URL")
embedding_model = OpenAIEmbeddings()
# 2.加载HTML内容为一个文档对象
loader = WebBaseLoader("https://zh.wikipedia.org/wiki/%E7%8C%AB")
docs = loader.load()
#print(docs)
# 3.分割文档
splitter = RecursiveCharacterTextSplitter(
chunk_size=1000,
chunk_overlap=200
)
documents = splitter.split_documents(docs)
# 4.向量化 得到向量数据库对象
vector = FAISS.from_documents(documents, embedding_model)
# 5.创建检索器
retriever = vector.as_retriever()
# 测试检索结果
print(retriever.invoke("猫的特征")[0])

page_content='感官[编辑]
貓的感官適於狩獵。在哺乳類動物中，貓的聽覺、視覺、嗅覺、味覺、觸覺極敏銳。

視覺[编辑]
貓的瞳孔能縮得如線般狹小
貓的瞳孔變化
貓在晝間視覺縱不及人類，夜視能力與追蹤視覺上之活動物件卻相當出色，夜視能力是人類的六倍，雖然綜合色彩計算整體視覺系數則僅及人類的十分之一。貓的眼睛具有微光觀察能力，即使只有微弱月光都可分辨物件，有關光線入貓眼後可放大40至50倍處理，令貓具有夜間活動的能力，即使在黑暗的地下室貓咪依然能活動自如。強光下，貓會將瞳孔縮得如線般狹小，以減少對視網膜的傷害，但視野會因而縮窄。由於貓眼具備高幀與高分辨率視覺，故此電視機極微細之動靜對貓而言亦成逐格跳躍之畫面。[53]
另外，不只是光線會影響貓的瞳孔，感情也會。一般而言貓放鬆的時候瞳孔會縮小，而緊張時會放大。
貓的視網膜背面有一層藍綠色如熒光一般的薄膜（Tapetum Lucidum），可增加在暗處的視力。閃光中，貓眼能呈現各式各樣顔色。如同多數食肉動物，貓眼長在臉上朝正前方，賦予其遼闊的視野，單一貓眼視野為100度，雙眼視野為285度，加上頸旋靈活，總計視野比人類雙眼僅100度靈活得多。不過，貓僅能聚焦其前30厘米至3米之物件，相對人類而言屬大近視。
貓對三原色的辨識力很差，相對人類而言，貓有黃藍色盲且紅綠色辨成灰黃色，與狗相似。關於貓的夜視能力，生物學家發現牛磺酸對貓的視力起了很大作用，貓本身不能合成牛磺酸，必須由外攝取。缺乏牛磺酸，會使視力及夜視能力變差，影響夜間活動。據生物學家稱，貓經常捕鼠，是因老鼠體內含牛磺酸。[54][55]
紫外線可以通過貓的晶状体。
當四周光線微弱，貓會用感覺毛來改善行動力與感知能力。感覺毛主要分布於鼻子兩側、下巴、雙眼上方、兩頰也有數根。感覺毛可感受非常微弱的空氣波動，視野不清時也能協助辨識阻礙位置。鬍鬚尖端與雙耳連成一線，恰是身體能通過障礙的最小範圍，故貓可在黑夜中快速判斷地形能否通過。
貓有第三眼瞼，當貓眼睑張開時，眨眼時第三眼瞼會從旁稍微遮蓋眼睛。若貓生病，或是睡眠，笑著，此眼皮會縮回一部分。若貓長時間嶄露第三眼瞼，表示它的健康有問題。' metadata={'source': 'https://zh.wikipedia.org/wiki/%E7%8C%AB', 'title': '猫 - 维基百科，自由的百科全书'

3.创建工具、工具集

In [5]:
from langchain.tools.retriever import create_retriever_tool
# 创建一个工具来检索文档
retriever_tool = create_retriever_tool(
    retriever=retriever,
    name="wiki_search",
    description="搜索维基百科",
)
tools = [search, retriever_tool]

4.大模型的调用

In [6]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
# 获取大模型
model = ChatOpenAI(model="gpt-4o-mini")
# 模型绑定工具
model_with_tools = model.bind_tools(tools)
# 根据输入自动调用工具
messages = [HumanMessage(content="今天上海天气怎么样")]
response = model_with_tools.invoke(messages)
print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': '上海天气'}, 'id': 'call_Qedfw5rsNBcsZAYWiz4BSH0T', 'type': 'tool_call'}]


5.创建agent

In [10]:
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")
print(prompt.messages)
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor
# 创建Agent对象
agent = create_tool_calling_agent(model, tools, prompt)
# 创建AgentExecutor对象
agent_executor = AgentExecutor(agent=agent, tools=tools,verbose=True)
print(agent_executor.invoke({"input": "猫的特征"}))

/opt/anaconda3/envs/langchain_env/lib/python3.10/site-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}), MessagesPlaceholder(variable_name='chat_history', optional=True), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}), MessagesPlaceholder(variable_name='agent_scratchpad')]


> Entering new AgentExecutor chain...

Invoking: `wiki_search` with `{'query': '猫的特征'}`


感官[编辑]
貓的感官適於狩獵。在哺乳類動物中，貓的聽覺、視覺、嗅覺、味覺、觸覺極敏銳。

視覺[编辑]
貓的瞳孔能縮得如線般狹小
貓的瞳孔變化
貓在晝間視覺縱不及人類，夜視能力與追蹤視覺上之活動物件卻相當出色，夜視能力是人類的六倍，雖然綜合色彩計算整體視覺系數則僅及人類的十分之一。貓的眼睛具有微光觀察能力，即使只有微弱月光都可分辨物件，有關光線入貓眼後可放大40至50倍處理，令貓具有夜間活動的能力，即使在黑暗的地下室貓咪依然能活動自如。強光下，貓會將瞳孔縮得如線般狹小，以減少對視網膜的傷害，但視野會因而縮窄。由於貓眼具備高幀與高分辨率視覺，故此電視機極微細之動靜對貓而言亦成逐格跳躍之畫面。[53]
另外，不只是光線會影響貓的瞳孔，感情也會。一般而言貓放鬆的時候瞳孔會縮小，而緊張時會放大。
貓的視網膜背面有一層藍綠色如熒光一般的薄膜（Tapetum Lucidum），可增加在暗處的視力。閃光中，貓眼能呈現各式各樣顔色。如同多數食肉動物，貓眼長在臉上朝正前方，賦予其遼闊的視野，單一貓眼視野為100度，

6.添加记忆

In [20]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
store = {}
# 调取指定session_id对应的memory
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]
agent_with_chat_history = RunnableWithMessageHistory(
    runnable=agent_executor,
    get_session_history=get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)
response = agent_with_chat_history.invoke(
    {"input": "Hi，我的名字是Cyber"},
    config={"configurable": {"session_id": "123"}},
)
print(response)
response1 = agent_with_chat_history.invoke(
    {"input": "我叫什么名字?"},
    config={"configurable": {"session_id": "123"}},
    )
print(response1)



> Entering new AgentExecutor chain...
你好，Cyber！很高兴认识你。你今天想聊些什么？

> Finished chain.
{'input': 'Hi，我的名字是Cyber', 'chat_history': [], 'output': '你好，Cyber！很高兴认识你。你今天想聊些什么？'}


> Entering new AgentExecutor chain...
你的名字是Cyber。你想聊聊自己的名字吗，还是有其他问题？

> Finished chain.
{'input': '我叫什么名字?', 'chat_history': [HumanMessage(content='Hi，我的名字是Cyber', additional_kwargs={}, response_metadata={}), AIMessage(content='你好，Cyber！很高兴认识你。你今天想聊些什么？', additional_kwargs={}, response_metadata={})], 'output': '你的名字是Cyber。你想聊聊自己的名字吗，还是有其他问题？'}


In [21]:
response1 = agent_with_chat_history.invoke(
    {"input": "我叫什么名字?"},
    config={"configurable": {"session_id": "123"}},
    )
print(response1)



> Entering new AgentExecutor chain...
你的名字是Cyber。请问还有什么我可以帮助你的？

> Finished chain.
{'input': '我叫什么名字?', 'chat_history': [HumanMessage(content='Hi，我的名字是Cyber', additional_kwargs={}, response_metadata={}), AIMessage(content='你好，Cyber！很高兴认识你。你今天想聊些什么？', additional_kwargs={}, response_metadata={}), HumanMessage(content='我叫什么名字?', additional_kwargs={}, response_metadata={}), AIMessage(content='你的名字是Cyber。你想聊聊自己的名字吗，还是有其他问题？', additional_kwargs={}, response_metadata={})], 'output': '你的名字是Cyber。请问还有什么我可以帮助你的？'}
